In [1]:
%matplotlib notebook
%matplotlib inline

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
import pandas as pd
from collections import defaultdict

In [2]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
from PIL import Image
import os
import random

In [4]:
val_anno=load_json(os.path.join(OK_PATH,'mscoco_val2014_annotations.json'))['annotations']
ans_dict={}
for row in val_anno:
    ques_id=str(row['question_id'])
    answers=defaultdict(int)
    for info in row['answers']:
        answers[info['answer']]+=1
    #ans={ans:answers[ans]/sum(answers.values()) for ans in answers.keys()}
    ans_dict[ques_id]=answers
print (len(val_anno),len(ans_dict))

5046 5046


In [5]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [6]:
vqa_all=load_json(os.path.join(OK_PATH,'OpenEnded_mscoco_val2014_questions.json'))['questions']
print (len(vqa_all))

5046


In [7]:
def get_score(occurences):
    if occurences == 0:
        return 0
    elif occurences == 1:
        return 0.3
    elif occurences == 2:
        return 0.6
    elif occurences == 3:
        return 0.9
    else:
        return 1

In [8]:
ques_cap=load_pkl("""
CAPTION_PATH
""")

In [9]:
import inflect
single = inflect.engine()

def norm_ans(ans):
    if ans.endswith(',') or ans.endswith('.'):
        ans=ans[:-1]
    if ans.startswith('a '):
        ans=ans[2:]
    if ',' in ans:
        ans=ans.split(',')[0]
    if len(ans.split(' '))==1 and ans not in ['grass','glass']:
        norm_ans=single.singular_noun(ans)
        if norm_ans !=False:
            ans=norm_ans
    return ans

In [10]:
import openai

In [11]:
#engine_name='code-davinci-002'
engine_name='text-davinci-003'
temperature=0.0
max_tokens=5
top_p=0.75
frequency_penalty=0.0
presence_penalty=0.0
num_sequence=1
openai.api_key =open('../OK_VQA/openai_key.txt','r').readlines()[0].strip()

In [12]:
#engine_name='code-davinci-002'
engine_name='text-davinci-003'
temperature=0.0
max_tokens=5
top_p=0.75
frequency_penalty=0.0
presence_penalty=0.0
num_sequence=1
openai.api_key =open('../OK_VQA/mine.txt','r').readlines()[0].strip()

In [13]:
head='Please answer the question accordint to the above context with as few words as possible'

In [38]:
vis=0
acc=0.0
#random.shuffle(vqa_all)
print (len(vqa_all))
invalid=[]
total={}
for k,row in enumerate(vqa_all):
    if vis>500:
        break
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    if os.path.exists(os.path.join('../OK_VQA/cluster_generated_kb',
                                   ques_id+'.json'))==False:
        continue
    ques=row['question']
    cap=ques_cap[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',ques_id+'.json'))
    inputs=[head+'\n']
    inputs.append('Context:'+cap)
    inputs.append('Question:'+ques)
    inputs.append('Answer:')
    inputs='\n'.join(inputs)
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=inputs,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            #top_p=top_p,
                                            #logprobs=1,
                                            #frequency_penalty=frequency_penalty,
                                            #presence_penalty=presence_penalty,
                                            stream=False,
                                            stop=["\n"]
                                           )
        pred=response["choices"][0]["text"].strip().lower()
    except:
        invalid.append(ques_id)
        vis+=1
        print (ques_id)
        continue
    ans=ans_dict[ques_id]
        
    pred=norm_ans(pred)
    total[ques_id]=pred
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
        
    """print(ques,ans)
    print(cap)
    print(pred)
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    display(im)"""
    vis+=1
print (acc*100.0/vis)

5046
Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 30.857142857142865
Already finished: 400 7.927070947284978

	Acc: 29.714285714285722
Already finished: 600 11.890606420927467

	Acc: 27.929515418502206
Already finished: 800 15.854141894569956

	Acc: 28.184818481848186
Already finished: 1000 19.817677368212447

	Acc: 26.595744680851055
Already finished: 1200 23.781212841854934

	Acc: 26.377440347071566
27.38522954091815


In [40]:
pkl.dump(total,open('../EACL-24-results/zero-gpt-500.pkl','wb'))

In [12]:
head='Please answer the question accordint to the above context and knowledge with as few words as possible'

In [43]:
vis=0
acc=0.0
NUM_KB=10
#random.shuffle(vqa_all)
print (len(vqa_all))
invalid=[]
total={}
for k,row in enumerate(vqa_all):
    if vis>500:
        break
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    if os.path.exists(os.path.join('../OK_VQA/cluster_generated_kb',
                                   ques_id+'.json'))==False:
        continue
    ques=row['question']
    cap=ques_cap[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',ques_id+'.json'))
    kb=kb[:NUM_KB]
    kb=' . '.join(kb)
    inputs=[head+'\n']
    inputs.append('Context:'+cap)
    inputs.append('Knowledge:'+kb)
    inputs.append('Question:'+ques)
    inputs.append('Answer:')
    inputs='\n'.join(inputs)
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=inputs,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            #top_p=top_p,
                                            #logprobs=1,
                                            #frequency_penalty=frequency_penalty,
                                            #presence_penalty=presence_penalty,
                                            stream=False,
                                            stop=["\n"]
                                           )
        pred=response["choices"][0]["text"].strip().lower()
    except:
        invalid.append(ques_id)
        vis+=1
        print (ques_id)
        continue
    ans=ans_dict[ques_id]
        
    pred=norm_ans(pred)
    total[ques_id]=pred
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
        
    """print(ques,ans)
    print(cap)
    print(pred)
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    display(im)"""
    vis+=1
print (acc*100.0/vis)

5046
Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 34.857142857142854
Already finished: 400 7.927070947284978

	Acc: 31.714285714285722
Already finished: 600 11.890606420927467

	Acc: 32.2466960352423
4570785
Already finished: 800 15.854141894569956

	Acc: 31.88118811881188
Already finished: 1000 19.817677368212447

	Acc: 31.063829787234035
Already finished: 1200 23.781212841854934

	Acc: 31.843817787418633
32.41516966067862


In [44]:
pkl.dump(total,open('../EACL-24-results/kb_10_zero-gpt-500.pkl','wb'))

In [45]:
vis=0
acc=0.0
NUM_KB=20
#random.shuffle(vqa_all)
print (len(vqa_all))
invalid=[]
total={}
for k,row in enumerate(vqa_all):
    if vis>500:
        break
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    if os.path.exists(os.path.join('../OK_VQA/cluster_generated_kb',
                                   ques_id+'.json'))==False:
        continue
    ques=row['question']
    cap=ques_cap[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',ques_id+'.json'))
    kb=kb[:NUM_KB]
    kb=' . '.join(kb)
    inputs=[head+'\n']
    inputs.append('Context:'+cap)
    inputs.append('Knowledge:'+kb)
    inputs.append('Question:'+ques)
    inputs.append('Answer:')
    inputs='\n'.join(inputs)
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=inputs,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            #top_p=top_p,
                                            #logprobs=1,
                                            #frequency_penalty=frequency_penalty,
                                            #presence_penalty=presence_penalty,
                                            stream=False,
                                            stop=["\n"]
                                           )
        pred=response["choices"][0]["text"].strip().lower()
    except:
        invalid.append(ques_id)
        vis+=1
        print (ques_id)
        continue
    ans=ans_dict[ques_id]
        
    pred=norm_ans(pred)
    total[ques_id]=pred
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
        
    """print(ques,ans)
    print(cap)
    print(pred)
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    display(im)"""
    vis+=1
print (acc*100.0/vis)

5046
Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 31.428571428571434
Already finished: 400 7.927070947284978

	Acc: 29.2857142857143
Already finished: 600 11.890606420927467

	Acc: 30.13215859030838
Already finished: 800 15.854141894569956

	Acc: 30.495049504950494
Already finished: 1000 19.817677368212447

	Acc: 30.478723404255305
Already finished: 1200 23.781212841854934

	Acc: 31.32321041214749
31.736526946107766


In [46]:
pkl.dump(total,open('../EACL-24-results/kb_20_zero-gpt-500.pkl','wb'))

In [47]:
vis=0
acc=0.0
NUM_KB=5
#random.shuffle(vqa_all)
print (len(vqa_all))
invalid=[]
total={}
for k,row in enumerate(vqa_all):
    if vis>500:
        break
    if k%200==0:
        print ('Already finished:',
               k,k*100.0/len(vqa_all) )
        if k>0:
            print ('\n\tAcc:',acc*100.0/vis)
    ques_id=str(row['question_id'])
    if os.path.exists(os.path.join('../OK_VQA/cluster_generated_kb',
                                   ques_id+'.json'))==False:
        continue
    ques=row['question']
    cap=ques_cap[ques_id]
    kb=load_json(os.path.join('../OK_VQA/cluster_generated_kb',ques_id+'.json'))
    kb=kb[:NUM_KB]
    kb=' . '.join(kb)
    inputs= [head+'\n']
    inputs.append('Context:'+cap)
    inputs.append('Knowledge:'+kb)
    inputs.append('Question:'+ques)
    inputs.append('Answer:')
    inputs='\n'.join(inputs)
    try:
        response = openai.Completion.create(engine=engine_name,
                                            prompt=inputs,
                                            temperature=temperature,
                                            max_tokens=max_tokens,
                                            #top_p=top_p,
                                            #logprobs=1,
                                            #frequency_penalty=frequency_penalty,
                                            #presence_penalty=presence_penalty,
                                            stream=False,
                                            stop=["\n"]
                                           )
        pred=response["choices"][0]["text"].strip().lower()
    except:
        invalid.append(ques_id)
        vis+=1
        print (ques_id)
        continue
    ans=ans_dict[ques_id]
        
    pred=norm_ans(pred)
    total[ques_id]=pred
    if pred in ans_dict[ques_id]:
        acc+=get_score(ans_dict[ques_id][pred])
        
    """print(ques,ans)
    print(cap)
    print(pred)
    img_id='COCO_val2014_'+str(row['image_id']).zfill(12)+'.jpg'
    im=Image.open(os.path.join(OK_PATH,'images',img_id)).convert('RGB')
    display(im)"""
    vis+=1
print (acc*100.0/vis)

5046
Already finished: 0 0.0
Already finished: 200 3.963535473642489

	Acc: 37.42857142857144
Already finished: 400 7.927070947284978

	Acc: 32.71428571428572
Already finished: 600 11.890606420927467

	Acc: 33.12775330396477
Already finished: 800 15.854141894569956

	Acc: 34.059405940594054
Already finished: 1000 19.817677368212447

	Acc: 33.56382978723402
3548325
Already finished: 1200 23.781212841854934

	Acc: 33.44902386117134
34.09181636726544


In [48]:
pkl.dump(total,open('../EACL-24-results/kb_5_zero-gpt-500.pkl','wb'))